# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /Users/zhili/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/zhili/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/zhili/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# import libraries
import re
import pandas as pd
import numpy as np
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pprint

In [4]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('DisasterTable',engine)
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [5]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('Vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10))),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 42)

# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('Vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [8]:
# calculate f1 score, precision and recall for each output category of dataset
for i, col in enumerate(y_test):
        print("Feature: {}".format(col))
        print(classification_report(y_test[col], y_pred[:, i]))

Feature: related
              precision    recall  f1-score   support

           0       0.61      0.35      0.45      2054
           1       0.81      0.93      0.87      6534
           2       0.67      0.06      0.11        64

    accuracy                           0.79      8652
   macro avg       0.70      0.45      0.48      8652
weighted avg       0.76      0.79      0.76      8652

Feature: request
              precision    recall  f1-score   support

           0       0.88      0.99      0.93      7180
           1       0.84      0.35      0.49      1472

    accuracy                           0.88      8652
   macro avg       0.86      0.67      0.71      8652
weighted avg       0.87      0.88      0.86      8652

Feature: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8614
           1       0.00      0.00      0.00        38

    accuracy                           1.00      8652
   macro avg       0.50  

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8467
           1       0.75      0.02      0.03       185

    accuracy                           0.98      8652
   macro avg       0.86      0.51      0.51      8652
weighted avg       0.97      0.98      0.97      8652

Feature: tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      8599
           1       0.00      0.00      0.00        53

    accuracy                           0.99      8652
   macro avg       0.50      0.50      0.50      8652
weighted avg       0.99      0.99      0.99      8652

Feature: hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      8567
           1       0.00      0.00      0.00        85

    accuracy                           0.99      8652
   macro avg       0.50      0.50      0.50      8652
weighted avg       0.98      0.99      0

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('Vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a232b2488>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_features='auto',
           

In [10]:
parameters = {
    'clf__estimator__max_features': ['auto', 'sqrt', 'log2'],
    'clf__estimator__n_estimators': [10, 50, 100],
    'clf__estimator__min_samples_leaf': [2, 5, 10]
}

cv = GridSearchCV(pipeline,param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
cv.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('Vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [12]:
y_pred_cv = cv.predict(X_test)

In [14]:
print(cv.best_params_)

{'clf__estimator__max_features': 'sqrt', 'clf__estimator__min_samples_leaf': 2, 'clf__estimator__n_estimators': 100}


In [13]:
for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred_cv[:, i]))

related
              precision    recall  f1-score   support

           0       0.75      0.18      0.29      2054
           1       0.79      0.98      0.87      6534
           2       1.00      0.02      0.03        64

    accuracy                           0.78      8652
   macro avg       0.85      0.39      0.40      8652
weighted avg       0.78      0.78      0.73      8652

request
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      7180
           1       0.94      0.29      0.44      1472

    accuracy                           0.88      8652
   macro avg       0.91      0.64      0.69      8652
weighted avg       0.88      0.88      0.85      8652

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8614
           1       0.00      0.00      0.00        38

    accuracy                           1.00      8652
   macro avg       0.50      0.50      0.50      865

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      8599
           1       0.00      0.00      0.00        53

    accuracy                           0.99      8652
   macro avg       0.50      0.50      0.50      8652
weighted avg       0.99      0.99      0.99      8652

hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      8567
           1       0.00      0.00      0.00        85

    accuracy                           0.99      8652
   macro avg       0.50      0.50      0.50      8652
weighted avg       0.98      0.99      0.99      8652

shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8618
           1       0.00      0.00      0.00        34

    accuracy                           1.00      8652
   macro avg       0.50      0.50      0.50      8652
weighted avg       0.99      1.00      0.99      8652

aid

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [6]:
# Define new feature StartingVerbExtractor
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [11]:
# define new_pipeline mode
new_pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('Vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', KNeighborsClassifier())
])

In [13]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 42)

# train new classifier
model = new_pipeline.fit(X_train, y_train)

In [14]:
# predict on test data with new_model
y_pred = new_pipeline.predict(X_test)

In [15]:
# evaluate the performance of new classifier
for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.55      0.40      0.47      2054
           1       0.82      0.90      0.86      6534
           2       0.78      0.22      0.34        64

    accuracy                           0.77      8652
   macro avg       0.72      0.51      0.56      8652
weighted avg       0.76      0.77      0.76      8652

request
              precision    recall  f1-score   support

           0       0.88      0.97      0.93      7180
           1       0.75      0.37      0.50      1472

    accuracy                           0.87      8652
   macro avg       0.82      0.67      0.71      8652
weighted avg       0.86      0.87      0.85      8652

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8614
           1       0.00      0.00      0.00        38

    accuracy                           1.00      8652
   macro avg       0.50      0.50      0.50      865

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8467
           1       0.67      0.02      0.04       185

    accuracy                           0.98      8652
   macro avg       0.82      0.51      0.52      8652
weighted avg       0.97      0.98      0.97      8652

tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      8599
           1       0.00      0.00      0.00        53

    accuracy                           0.99      8652
   macro avg       0.50      0.50      0.50      8652
weighted avg       0.99      0.99      0.99      8652

hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      8567
           1       0.00      0.00      0.00        85

    accuracy                           0.99      8652
   macro avg       0.50      0.50      0.50      8652
weighted avg       0.98      0.99      0.99      8652

sho

### 9. Export your model as a pickle file

In [16]:
filename = 'classifier.pkl'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.